# Loading Dataset From kaggle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

# Upload the kaggle.json file
files.upload()

# Move kaggle.json to the .kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k